# Text Classifier with the Interpreter API

This notebook shows you how to use the Interpreter API to classify text.

## Preparation

Let's start by installing the TensorFlow Model Garden library.



In [ ]:
pip install -U -q "tf-models-official==2.16.*"

Next we download an off-the-shelf model. Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/text/text_classifier#models) for more text classification models that you can use.

In [ ]:
!wget -O bert_classifier.tflite -q https://storage.googleapis.com/mediapipe-models/text_classifier/bert_classifier/float32/1/bert_classifier.tflite

In [ ]:
# @markdown Choose a text classification model
MODEL_PATH = "bert_classifier.tflite" # @param ["bert_classifier.tflite"]


Optionally, you can upload your own model (.tflite). If you want to do so, uncomment and run the cell below.


In [ ]:
# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)

# MODEL_PATH = list(uploaded.keys())[0]

# print('Uploaded model:', MODEL_PATH)

Read the associated files from models
The TensorFlow Lite model with metadata and associated files is essentially a zip file that can be unpacked with common zip tools to get the associated files. For example, you can unzip **1.tflite** and extract vocab.txt in the model as follows:


In [ ]:
import zipfile

VOCAB_FILE = 'vocab.txt'

with open(VOCAB_FILE, 'wb') as f:
  vocab_data = zipfile.ZipFile(MODEL_PATH).open(VOCAB_FILE).read()
  f.write(vocab_data)

## Load the model using the Interpreter API

Here you'll be loading the model just so we can save the input shape for future use during tokenization.

In [ ]:
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get the input shape
input_shape = input_details[0]['shape']
input_shape

## Tokenizer
To run inference using a pre-trained language model such as BERT, you need to make sure that you're using exactly the same tokenization, vocabulary, and index mapping as used during training.

The following code rebuilds the tokenizer that was used by the base model using the Model Garden's `tfm.nlp.layers.FastWordpieceBertTokenizer` layer:

In [ ]:
import tensorflow_models as tfm

do_lower_case = True

# Define a tokenizer
tokenizer = tfm.nlp.layers.FastWordpieceBertTokenizer(vocab_file=VOCAB_FILE, lower_case=do_lower_case)

Let's tokenize a test sentence:



In [ ]:
text = ["I'm looking forward to it."]
tokenized_text = tokenizer(text)

## Pack the inputs

TensorFlow Model Garden's `tfm.nlp.layers.BertPackInputs` layer can handle the conversion from a list of tokenized sentences to the input format expected by the BERT model. This layer packs the two input sentences concatenated together. This input is expected to start with a [CLS] "This is a classification problem" token, and each sentence should end with a [SEP] "Separator" token. It also needs to know the indices of the tokenizer's special tokens.

In [ ]:
special = tokenizer.get_special_tokens_dict()
special

In [ ]:
# Ensure the max seq length matches the model input shape
max_seq_length = input_shape[1]

packer = tfm.nlp.layers.BertPackInputs(
    seq_length=max_seq_length,
    special_tokens_dict=tokenizer.get_special_tokens_dict())

Let's try the preprocessing model on some text and see the output:



In [ ]:
packed_inputs = packer(tokenized_text)

Then, it returns a dictionary containing three outputs:

* input_word_ids: The tokenized sentences packed together.  
* input_mask: The mask indicating which locations are valid in the other
outputs.  
* input_type_ids: Indicating which sentence each token belongs to.

In [ ]:
print(f'Keys       : {list(packed_inputs.keys())}')
print(f'Shape      : {packed_inputs["input_word_ids"].shape}')
print(f'Word Ids   : {packed_inputs["input_word_ids"][0, :12]}')
print(f'Input Mask : {packed_inputs["input_mask"][0, :12]}')
print(f'Type Ids   : {packed_inputs["input_type_ids"][0, :12]}')

Put it all together
Combine these two parts into a `keras.layers.Layer` that can be used during inference.


In [ ]:
class BertInputProcessor(tf.keras.layers.Layer):
  def __init__(self, tokenizer, packer):
    super().__init__()
    self.tokenizer = tokenizer
    self.packer = packer

  def call(self, inputs):
    tokenized = self.tokenizer(inputs)
    packed = self.packer(tokenized)
    return packed

## Running inference

Here are the steps to run text classification using the Interpreter API.

Check out the [Interpreter documentation](https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python) to learn more about configuration options for the Interpreter API.

In [ ]:
INPUT_TEXT = "It's been a great day" # @param {type:"string"}

In [ ]:
# STEP 1: Import the necessary modules.
import numpy as np

# STEP 2: Create a text preprocessor.
bert_inputs_processor = BertInputProcessor(tokenizer, packer)

# STEP 3: Set the input tensors and perform text classification on the input.
packed_inputs = bert_inputs_processor(tf.constant([INPUT_TEXT]))
interpreter.set_tensor(input_details[0]["index"], packed_inputs['input_word_ids'].numpy())
interpreter.set_tensor(input_details[1]["index"], packed_inputs['input_type_ids'].numpy())
interpreter.set_tensor(input_details[2]["index"], packed_inputs['input_mask'].numpy())
interpreter.invoke()

# # STEP 4: Process the classification result. In this case, print out the most likely category.
output_tensor = interpreter.get_tensor(output_details[0]['index'])
classification_probability = np.argmax(output_tensor)
print('Positive' if classification_probability == 1 else 'Negative')